In [15]:
# Use this cell when you modify the recsys package and need to reload it
import importlib
import sys

# Reload the recsys modules
if 'recsys' in sys.modules:
    importlib.reload(sys.modules['recsys.utils.files'])
    importlib.reload(sys.modules['recsys.utils'])
    print("Modules reloaded successfully")
else:
    print("Modules not yet loaded")

Modules reloaded successfully


# Collaborative Filtering using Cosine similarity


# Importing required packages
We start off by defining our root project entry point

In [16]:
import sys
from pathlib import Path
project_root = Path("..").resolve()
if str(project_root / "src") not in sys.path:
    sys.path.append(str(project_root / "src"))

This next code snippet unzips the .zip file in "data/raw" and saves it in "data/processed" folders respectively

In [17]:
from recsys.utils import unzip_file
zip_path = project_root / "data" / "raw" / "ml-latest-small.zip"
destination = project_root / "data" / "processed"

unzip_file(zip_path, destination, overwrite=True)

PosixPath('/Users/tonyli/Documents/Projects/craftyverse/craftyverse-recsys/data/processed')

In [18]:

from recsys.utils import read_csv
csv_path = project_root / "data" /"processed" / "ml-latest-small" / "ratings.csv"

ratings_df = read_csv(csv_path)
print(ratings_df.head())



   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


## Create user-item ratings matrix
Pivot ratings so rows are movieId and columns are userId.

In [19]:
ratings_matrix = ratings_df.pivot_table(
    index='userId',
    columns='movieId',
    values='rating',
    fill_value=0
)
ratings_matrix = ratings_matrix.sort_index().sort_index(axis=1)
print(ratings_matrix)


movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
606         2.5     0.0     0.0     0.0     0.0     0.0     2.5     0.0   
607         4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
608         2.5     2.0     2.0     0.0     0.0     0.0     0.0     0.0   
609         3.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
610         5.0     0.0     0.0     0.0     0.0     5.0     0.0     0.0   

movieId  9       10     

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

user_similarity = cosine_similarity(ratings_matrix)
userIds = ratings_matrix.index.tolist()

user_similarity_df = pd.DataFrame(user_similarity,
                                  index=userIds,
                                  columns=userIds)

print(user_similarity_df)

          1         2         3         4         5         6         7    \
1    1.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
2    0.027283  1.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
3    0.059720  0.000000  1.000000  0.002251  0.005020  0.003936  0.000000   
4    0.194395  0.003726  0.002251  1.000000  0.128659  0.088491  0.115120   
5    0.129080  0.016614  0.005020  0.128659  1.000000  0.300349  0.108342   
..        ...       ...       ...       ...       ...       ...       ...   
606  0.164191  0.028429  0.012993  0.200395  0.106435  0.102123  0.200035   
607  0.269389  0.012948  0.019247  0.131746  0.152866  0.162182  0.186114   
608  0.291097  0.046211  0.021128  0.149858  0.135535  0.178809  0.323541   
609  0.093572  0.027565  0.000000  0.032198  0.261232  0.214234  0.090840   
610  0.145321  0.102427  0.032119  0.107683  0.060792  0.052668  0.193219   

          8         9         10   ...       601       602       603  \
1  

In [22]:
from recsys.recommendation import CF_cosine_recommender
recommended_items = CF_cosine_recommender(1, user_similarity_df, ratings_matrix)
print(recommended_items)

[356, 296, 318, 2571, 260]
